## JIRA metrics
This is a juypter notebook to walk through the steps required to pull down Jira sprint metrics

In [28]:
# these are your imports
import os

from jira import JIRA
import pandas as pd

In [29]:
# Configure JIRA python client and login with your
# API token
username = os.environ['JIRA_USERNAME']
api_token = os.environ['JIRA_API_TOKEN']
jira_client = JIRA(
    server='https://sagebionetworks.jira.com/',
    basic_auth=(username, api_token)
)

In [30]:
# Get all sprints
# Board 189 is the DPE scrum board
all_sprints = jira_client.sprints(board_id=189)
for sprint in all_sprints:
    if sprint.name.startswith(("ETL", "Orca", "DPE")) and "Sprint" not in sprint.name:
        print(sprint)

DPE 11.07.22 - 11.18.22
Orca 11.21.22 - 12.05.22
Orca 11.07.22 - 11.18.22
ETL 11.07.22 - 11.18.23
ETL 11.21.22 - 12.05.22
DPE 11.21.22 - 12.05.22
Orca 12.05.22 - 12.16.22
ETL 12.05.22 - 12.19.22
DPE 12.05.22 - 12.16.22
DPE 12.19.22 - 01.02.23
Orca 12.19.22 - 01.02.23
ETL 12.19.22 - 01.02.23
DPE 01.03.23 - 01.16.23
ETL 01.03.23 - 01.16.23
ETL 2023-01-17 to 2023-01-29
DPE 2023-01-17 to 2023-01-29
DPE 2023-01-30 to 2023-02-13
ETL 2023-01-30 to 2023-02-13
DPE 2023-02-13 to 2023-02-27
ETL 2023-02-13 to 2023-02-27
DPE 2023-02-27 to 2023-03-13
ETL 2023-02-27 to 2023-03-13
DPE 2023-03-13 to 2023-03-27
Orca 2023-03-13 to 2023-03-27
ETL 2023-03-13 to 2023-03-27
DPE 2023-03-27 to 2023-04-10


In [31]:
# Get tickets for a specific sprint
sprint_id = sprint.id
issues = jira_client.search_issues(f"sprint={sprint_id}")

In [32]:
# Get all issues in a sprint
# This does NOT take into account the specific status of
# an issue at the duration of the sprint
# For example, an issue could be "Waiting for review" at the
# end of the sprint, but can be "Closed" now. This will
# skew the "number of story points" per engineer over time.
result = []
for issue in issues:
    issue_info = jira_client.issue(issue.id)
    issue_assignee = issue_info.fields.assignee.displayName
    # Story points
    issue_story_points = issue_info.fields.customfield_10014
    # Status of ticket
    issue_status = issue_info.fields.status.name
    issue_summary = issue_info.fields.summary
    issue_desc = issue_info.fields.description
    issue_type_name = issue_info.fields.issuetype.name
    # Target start
    issue_start_date = issue_info.fields.customfield_12113
    issue_due_date = issue_info.fields.duedate

    result.append({
        'sprint_id': sprint_id,
        "issuetype": issue_type_name,
        "key": issue.id,
        "summary": issue_summary,
        "description": issue_desc,
        'status': issue_status,
        "assignee": issue_assignee,
        "sprint": sprint,
        "target_start": issue_start_date,
        "due_date": issue_due_date
    })
jira_issues_df = pd.DataFrame(result)



In [33]:

print(jira_issues_df)

    sprint_id issuetype    key  \
0         573       Bug  84823   
1         573     Story  85844   
2         573     Story  85843   
3         573     Story  85841   
4         573     Story  85128   
5         573     Story  84711   
6         573     Story  84631   
7         573     Story  84264   
8         573     Story  85858   
9         573     Story  85191   
10        573     Story  84460   
11        573     Story  83551   
12        573     Story  83550   
13        573     Story  81736   
14        573       Bug  85499   
15        573  Sub-Task  85959   
16        573     Story  85872   
17        573     Story  85507   
18        573     Story  85506   
19        573  Sub-Task  85505   
20        573       Bug  85474   
21        573     Story  85473   
22        573  Sub-Task  85462   
23        573     Story  85299   
24        573     Story  85210   
25        573     Story  82467   

                                              summary  \
0   Sarek v3 randomly sk